# Dynamic Control and Setpoint Changes

Time-domain simulations typically use fixed parameters throughout the simulation period. However, many advanced applications require changing control setpoints during simulation, such as implementing economic dispatch schedules, testing automatic generation control (AGC), or integrating with reinforcement learning algorithms. ANDES supports this through its multi-stage simulation capability.

The key insight is that `ss.TDS.run()` can be called multiple times, each time advancing the simulation from the current state to a new end time. Between calls, you can modify setpoints, reference values, or other parameters, and the simulation will continue from exactly where it left off.

This tutorial demonstrates how to change generator power setpoints during simulation and observe the dynamic response.

:::{note}
**Prerequisites:** Complete {doc}`04-time-domain` for time-domain simulation basics and {doc}`05-data-and-formats` for parameter modification techniques.
:::

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import andes

andes.config_logger(stream_level=20)

## Initialize the System

First, we load the system and run power flow to establish the initial operating point. We also disable any pre-existing disturbances (like the Toggle in the Kundur test case) so we can observe only the effects of our setpoint changes.

In [2]:
kundur = andes.get_case('kundur/kundur_full.xlsx')
ss = andes.run(kundur)

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.2737 seconds.


...

3: |F(x)| = 0.002038626956


4: |F(x)| = 3.745103979e-07


Converged in 5 iterations in 0.0025 seconds.


Report saved to "kundur_full_out.txt" in 0.0007 seconds.


-> Single process finished in 0.4709 seconds.


In [3]:
# Disable the pre-existing Toggle to have a clean experiment
ss.Toggle.alter('u', 1, 0)

## Understanding Turbine Governor Setpoints

The TGOV1 turbine governor model includes an auxiliary power input `paux0` that can be used to inject additional power commands. This is commonly used for AGC signals or economic dispatch adjustments. Let us examine the TGOV1 model structure to understand where this setpoint enters the equations.

In [4]:
# View the TGOV1 algebraic equations to see how paux0 is used
print("TGOV1 Algebraic Equations:")
for var_name in ['paux', 'pd']:
    var = getattr(ss.TGOV1, var_name)
    print(f"  {var_name}: {var.e_str}")

TGOV1 Algebraic Equations:
  paux: paux0 - paux
  pd: ue*(- wd + pref + paux) * gain - pd


The equation `paux0 - paux = 0` means that the algebraic variable `paux` tracks the parameter `paux0`. The `paux` variable is then added to the power reference signal in the `pd` equation, ultimately affecting the turbine mechanical power output.

Let us check the initial values of `paux0` for all generators.

In [5]:
print(f"Initial paux0 values: {ss.TGOV1.paux0.v}")

Initial paux0 values: [0. 0. 0. 0.]


## Stage 1: Run Simulation to Steady State

We first run the simulation for one second with no setpoint changes. This establishes a baseline and confirms the system is in steady state before we apply any changes.

In [6]:
# Set the first stopping time
ss.TDS.config.tf = 1
ss.TDS.run()

Initialization for dynamics completed in 0.0128 seconds.


Initialization was successful.


  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=1.00 sec completed in 0.0112 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0006 seconds.


True

## Stage 2: Apply Setpoint Change

Now we increase the auxiliary power setpoint for the first generator. This simulates a dispatch command instructing the generator to increase its output. The change can be made using either direct array assignment or the `alter()` method.

Important: When modifying parameter arrays directly, you must use in-place assignment (e.g., `array[0] = value`) rather than creating a new array. This ensures the change propagates correctly to the simulation.

In [7]:
# Increase the auxiliary power of the first generator by 0.05 pu
# Method 1: direct in-place assignment
ss.TGOV1.paux0.v[0] = 0.05

# Alternative Method 2: using alter()
# ss.TGOV1.alter('paux0', 1, 0.05)

print(f"Updated paux0 values: {ss.TGOV1.paux0.v}")

Updated paux0 values: [0.05 0.   0.   0.  ]


In [8]:
# Continue simulation to 2 seconds
ss.TDS.config.tf = 2
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=2.00 sec completed in 0.0310 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0013 seconds.


True

Let us plot the auxiliary power input to see the step change.

In [9]:
fig, ax = ss.TDS.plt.plot(ss.TGOV1.paux, ylabel='Auxiliary Power [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The plot shows that `paux` for the first generator stepped from 0 to 0.05 pu at t=1 second. This change propagates through the turbine governor dynamics to affect the mechanical power output.

In [10]:
fig, ax = ss.TDS.plt.plot(ss.TGOV1.pout, ylabel='Turbine Output Power [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The turbine output power increases for the first generator (blue curve) following the setpoint change. The response is not instantaneous because the turbine governor has dynamics (lag and lead-lag blocks) that smooth the response.

The generator speeds also respond to this power imbalance.

In [11]:
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega, ylabel='Generator Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Stage 3: Remove Setpoint Change

Now we reset the auxiliary power to zero and observe the system returning to its original operating point. This demonstrates that the simulation correctly tracks multiple setpoint changes.

In [12]:
# Reset the auxiliary power
ss.TGOV1.paux0.v[0] = 0.0

# Continue simulation to 10 seconds
ss.TDS.config.tf = 10
ss.TDS.run()

  0%|          | 0/100 [00:00<?, ?%/s]

Simulation to t=10.00 sec completed in 0.2250 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0086 seconds.


True

In [13]:
fig, ax = ss.TDS.plt.plot(ss.TGOV1.paux, ylabel='Auxiliary Power [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The auxiliary power shows two transitions: up at t=1s and back down at t=2s. The generator speeds show the corresponding dynamics as the system adjusts to each change.

In [14]:
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega, ylabel='Generator Speed [pu]')

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Applications

This multi-stage simulation capability enables several advanced applications:

| Application | Setpoint Modified | Use Case |
|-------------|-------------------|----------|
| Economic dispatch | `TGOV1.paux0` or `pref0` | Follow dispatch schedules |
| AGC simulation | `TGOV1.paux0` | Frequency regulation |
| Voltage control | `EXDC2.vref0` | Voltage setpoint tracking |
| Reinforcement learning | Various | AI-based control |
| Hardware-in-the-loop | Multiple | Real-time co-simulation |

The key technique is the same in all cases: run simulation to a checkpoint, modify the relevant parameter, then continue simulation.

## Cleanup

In [15]:
!andes misc -C

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.lst" removed.


## Next Steps

- {doc}`11-frequency-response` - Frequency response analysis
- {doc}`07-eigenvalue-analysis` - Small-signal stability analysis
- {doc}`08-parameter-sweeps` - Batch simulation techniques